In [14]:

import ast

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold


In [10]:

MODEL = 'paraphrase-mpnet-base-v2'

# Test different values of k
K_VALUES = [1, 3, 5, 7, 10, 15, 20, 30, 40, 50]

df = pd.read_csv('dataset/manga_dataset_with_embeddings_' + MODEL + '.csv')
df.head()

,title,type,score,genres,themes,demographics,authors,synopsis,background,url,embeddings
0,Berserk,manga,9.47,"['Action', 'Adventure', 'Award Winning', 'Dram...","['Gore', 'Military', 'Mythology', 'Psychologic...",['Seinen'],"[{'id': 1868, 'first_name': 'Kentarou', 'last_...","Guts, a former mercenary now known as the ""Bla...",Berserk won the Award for Excellence at the si...,https://myanimelist.net/manga/2/Berserk,"0.07373989,0.080003224,0.0119243795,0.04972718..."
1,One Piece,manga,9.22,"['Action', 'Adventure', 'Fantasy']",[],['Shounen'],"[{'id': 1881, 'first_name': 'Eiichiro', 'last_...","Gol D. Roger, a man referred to as the ""King o...",One Piece is the highest selling manga series ...,https://myanimelist.net/manga/13/One_Piece,"0.120224416,0.2653468,-0.006954093,0.023620032..."
2,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,manga,9.30,"['Action', 'Adventure', 'Mystery', 'Supernatur...",['Historical'],"['Seinen', 'Shounen']","[{'id': 2619, 'first_name': 'Hirohiko', 'last_...","In the American Old West, the world's greatest...",JoJo no Kimyou na Bouken Part 7: Steel Ball Ru...,https://myanimelist.net/manga/1706/JoJo_no_Kim...,"-0.13708049,0.2916467,0.015403955,0.18725339,0..."
3,Oyasumi Punpun,manga,9.02,"['Drama', 'Slice of Life']",['Psychological'],['Seinen'],"[{'id': 2836, 'first_name': 'Inio', 'last_name...",Punpun Onodera is a normal 11-year-old boy liv...,Oyasumi Punpun was originally serialized in Yo...,https://myanimelist.net/manga/4632/Oyasumi_Punpun,"0.14025311,0.3358102,-0.0030324818,0.071511015..."
4,Fullmetal Alchemist,manga,9.03,"['Action', 'Adventure', 'Award Winning', 'Dram...",['Military'],['Shounen'],"[{'id': 1874, 'first_name': 'Hiromu', 'last_na...",Alchemists are knowledgeable and naturally tal...,Hagane no Renkinjutsushi won the 49th Shogakuk...,https://myanimelist.net/manga/25/Fullmetal_Alc...,"0.16601335,0.24077635,0.018449467,0.18147695,0..."


# KNN - K-Fold-Cross-Validation

To find the best K value for KNN, we can use K Fold Cross Validation more robustly by reducing variability

We will also make train and test set to have our KNN perform predictions to see which k is optimal for making predictions amongst our k fold dataset

In [11]:
# Convert embeddings from string to list of floats
df['embeddings'] = df['embeddings'].apply(ast.literal_eval)

# Convert list of embeddings to numpy array for easy manipulation
embeddings = np.array(df['embeddings'].tolist())

# Extract ratings
ratings = df['score'].values

In [12]:
# Function to predict ratings using a given k
def knn_predict(X_train, y_train, X_val, k):
    similarity_matrix = cosine_similarity(X_val, X_train)
    predictions = []
    for idx, sim_scores in enumerate(similarity_matrix):
        similar_indices = np.argsort(sim_scores)[::-1][:k]
        similar_ratings = y_train[similar_indices]
        similar_scores = sim_scores[similar_indices]
        weighted_average = np.dot(similar_ratings, similar_scores) / similar_scores.sum()
        predictions.append(weighted_average)
    return np.array(predictions)

In [15]:
# Perform K-Fold-Cross-Validation to determine the best 'k'

# Known ratings mask
mask = ~pd.isnull(ratings)

# Data for cross-validation
X = embeddings[mask]
y = ratings[mask]

# Define k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

average_errors = []

# Perform k-fold cross-validation for each k
for k in K_VALUES:
    fold_errors = []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        y_pred = knn_predict(X_train, y_train, X_val, k)
        fold_error = mean_squared_error(y_val, y_pred)
        fold_errors.append(fold_error)
    
    average_error = np.mean(fold_errors)
    average_errors.append(average_error)
    print(f"k={k}, Average MSE={average_error}")

# Find the best k
best_k = K_VALUES[np.argmin(average_errors)]
print(f"Best k value: {best_k}")

k=1, Average MSE=0.4541213071925559
k=3, Average MSE=0.30167057620676974
k=5, Average MSE=0.27514267769931616
k=7, Average MSE=0.26475040891931334
k=10, Average MSE=0.25732609926002076
k=15, Average MSE=0.25165024357688004
k=20, Average MSE=0.2482493733208603
Best k value: 20


## KNN - Perform KNN to fill in missing ratings based on best 'k'

In [ ]:
# Compute cosine similarity matrix between all embeddings
similarity_matrix = cosine_similarity(embeddings)

In [8]:
def predict_rating(idx, k=best_k):
    """Computes the weighted average ratings for a null'd manga based on manga
    that are similar synopsis wise
    
    EXPLANATION for weighted average - 
    Multiply ratings by corresponding similarity scores then sums those products then divide by similarity score sum so we normalize the value
    
    Leverages the idea that manga with more similar synopsis should have a greater influence on the predicted ratings
    

    Args:
        idx (int): row index on dataframe
        k (int, optional): top k similar synposis. Defaults to 5.

    Returns:
        float: final rating for that manga without a rating
    """
    # Get the similarity scores for the target manga
    sim_scores = similarity_matrix[idx]
    
    # Get indices of k most similar manga with non-null ratings
    similar_indices = np.argsort(sim_scores)[::-1]
    similar_indices = [i for i in similar_indices if not pd.isnull(ratings[i])][:k]
    
    if not similar_indices:
        return np.nan  # If there are no similar rated manga, return NaN
    similar_ratings = ratings[similar_indices]
    similar_scores = sim_scores[similar_indices]
    
    # Compute the weighted average of the ratings
    return np.dot(similar_ratings, similar_scores) / similar_scores.sum()

In [ ]:
# Predict and fill missing ratings
for idx, rating in enumerate(ratings):
    if pd.isnull(rating):
        ratings[idx] = predict_rating(idx)

# Update the dataframe
df['score'] = ratings

In [ ]:
# Save the updated dataframe to a new CSV file
df.to_csv('manga_dataset_with_filled_ratings_' + MODEL + '.csv', index=False)